<a href="https://colab.research.google.com/github/faiqahmednizami/ai-health-assistant/blob/main/Copy_of_Lab_Multilevel_Pattern_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==== Cell 1: Install & Imports ====

!pip install openpyxl  # To make sure Excel reading works

import pandas as pd
import re
from itertools import combinations

In [ ]:
# ==== Cell 2: Upload Excel File ====
from google.colab import files

uploaded = files.upload()

# This will prompt you to choose the file from your computer.
# Make sure you select: Corporate Flown_Ticketed Data.xlsx

Saving Corporate Flown_Ticketed Data.xlsx to Corporate Flown_Ticketed Data (1).xlsx


In [ ]:
# ==== Cell 3: Load the DATA sheet ====

# Get the uploaded file name (first key in uploaded dict)
file_name = list(uploaded.keys())[0]
print("Using file:", file_name)

df = pd.read_excel(file_name, sheet_name="DATA")

print("Columns:", df.columns.tolist())
df.head()

Using file: Corporate Flown_Ticketed Data (1).xlsx
Columns: ['S. NO.', 'TICKET NO.', 'DATE', 'CITES', 'PAX NAME', 'AMOUNT']


,S. NO.,TICKET NO.,DATE,CITES,PAX NAME,AMOUNT
0,1.0,1.769373e+12,NaN,KHI-IAH/YYZ-KHI,AHMED/AHSAN MR,1600000.0
1,2.0,1.761006e+12,NaN,KHI-LON-KHI,ANWER / NIDA,1150000.0
2,3.0,1.761006e+12,NaN,LHR-DXB-KHI,ANWER NIDA,30000.0
3,4.0,1.766600e+12,NaN,KHI-DXB-KHI,ARIFF SANA,112000.0
4,5.0,1.766652e+12,NaN,DXB-KHI,ARIFF SANA,30000.0


In [ ]:
# ==== Cell 4: sort() demo (Ascending & Descending) ====

numbers = [12, 5, 9, 1, 7]

# Ascending
numbers.sort()
print("Ascending:", numbers)

# Descending
numbers2 = [12, 5, 9, 1, 7]
numbers2.sort(reverse=True)
print("Descending:", numbers2)

Ascending: [1, 5, 7, 9, 12]
Descending: [12, 9, 7, 5, 1]


In [ ]:
# ==== Cell 5: Keep only needed columns & drop missing ====

df = df[["CITES", "AMOUNT"]].dropna()
df.head()

,CITES,AMOUNT
0,KHI-IAH/YYZ-KHI,1600000.0
1,KHI-LON-KHI,1150000.0
2,LHR-DXB-KHI,30000.0
3,KHI-DXB-KHI,112000.0
4,DXB-KHI,30000.0


In [ ]:
# ==== Cell 6: Extract CITY_CODES from CITES ====

def extract_city_codes(cites_str):
    parts = re.split(r"[-/]", str(cites_str))
    codes = [p.strip().upper() for p in parts if p.strip()]
    # remove duplicates within same ticket
    return list(dict.fromkeys(codes))

df["CITY_CODES"] = df["CITES"].apply(extract_city_codes)

df[["CITES", "CITY_CODES"]].head()

,CITES,CITY_CODES
0,KHI-IAH/YYZ-KHI,"[KHI, IAH, YYZ]"
1,KHI-LON-KHI,"[KHI, LON]"
2,LHR-DXB-KHI,"[LHR, DXB, KHI]"
3,KHI-DXB-KHI,"[KHI, DXB]"
4,DXB-KHI,"[DXB, KHI]"


In [ ]:
# ==== Cell 7: Map city codes to regions (multilevel) ====

city_to_region = {
    # Pakistan
    "KHI": "Pakistan",
    "ISB": "Pakistan",

    # Middle East
    "DXB": "Middle East",
    "DOH": "Middle East",
    "JED": "Middle East",
    "MED": "Middle East",
    "RUH": "Middle East",

    # Europe / UK
    "LHR": "Europe/UK",
    "LGW": "Europe/UK",
    "LON": "Europe/UK",
    "AMS": "Europe/UK",
    "BER": "Europe/UK",
    "DUS": "Europe/UK",
    "BHX": "Europe/UK",

    # North America
    "IAH": "North America",
    "ORD": "North America",
    "YYZ": "North America",

    # Asia / Others
    "BOM": "South Asia",
    "BKK": "Asia/Others",
    "CGK": "Asia/Others",
    "DPS": "Asia/Others",
    "HKT": "Asia/Others",
    "KUL": "Asia/Others",
    "SEZ": "Asia/Others",
}

def map_to_regions(city_list):
    regions = set()
    for c in city_list:
        r = city_to_region.get(c, "Other")
        regions.add(r)
    return list(regions)

df["REGIONS"] = df["CITY_CODES"].apply(map_to_regions)

df[["CITES", "CITY_CODES", "REGIONS"]].head()

,CITES,CITY_CODES,REGIONS
0,KHI-IAH/YYZ-KHI,"[KHI, IAH, YYZ]","[Pakistan, North America]"
1,KHI-LON-KHI,"[KHI, LON]","[Pakistan, Europe/UK]"
2,LHR-DXB-KHI,"[LHR, DXB, KHI]","[Pakistan, Europe/UK, Middle East]"
3,KHI-DXB-KHI,"[KHI, DXB]","[Pakistan, Middle East]"
4,DXB-KHI,"[DXB, KHI]","[Pakistan, Middle East]"


In [ ]:
# ==== Cell 8: Create fare buckets (Low / Medium / High) ====

q1 = df["AMOUNT"].quantile(0.33)
q2 = df["AMOUNT"].quantile(0.66)

def fare_bucket(amount):
    if amount <= q1:
        return "Fare=Low"
    elif amount <= q2:
        return "Fare=Medium"
    else:
        return "Fare=High"

df["FARE_BUCKET"] = df["AMOUNT"].apply(fare_bucket)

print("Fare bucket thresholds:")
print("Low  <= ", q1)
print("Medium <= ", q2)
df[["AMOUNT", "FARE_BUCKET"]].head()

Fare bucket thresholds:
Low  <=  100444.90000000002
Medium <=  286629.36000000004


,AMOUNT,FARE_BUCKET
0,1600000.0,Fare=High
1,1150000.0,Fare=High
2,30000.0,Fare=Low
3,112000.0,Fare=Medium
4,30000.0,Fare=Low


In [ ]:
# ==== Cell 9: Build transactions for pattern mining ====

def build_items_level2(row):
    items = []
    for c in row["CITY_CODES"]:
        items.append(f"CITY={c}")
    items.append(row["FARE_BUCKET"])
    return items

def build_items_level1(row):
    items = []
    for r in row["REGIONS"]:
        items.append(f"REGION={r}")
    items.append(row["FARE_BUCKET"])
    return items

df["TRANS_L2"] = df.apply(build_items_level2, axis=1)
df["TRANS_L1"] = df.apply(build_items_level1, axis=1)

print("Sample Level 2 transactions:")
print(df["TRANS_L2"].head())

print("\nSample Level 1 transactions:")
print(df["TRANS_L1"].head())

Sample Level 2 transactions:
0    [CITY=KHI, CITY=IAH, CITY=YYZ, Fare=High]
1              [CITY=KHI, CITY=LON, Fare=High]
2     [CITY=LHR, CITY=DXB, CITY=KHI, Fare=Low]
3            [CITY=KHI, CITY=DXB, Fare=Medium]
4               [CITY=DXB, CITY=KHI, Fare=Low]
Name: TRANS_L2, dtype: object

Sample Level 1 transactions:
0    [REGION=Pakistan, REGION=North America, Fare=H...
1       [REGION=Pakistan, REGION=Europe/UK, Fare=High]
2    [REGION=Pakistan, REGION=Europe/UK, REGION=Mid...
3    [REGION=Pakistan, REGION=Middle East, Fare=Med...
4      [REGION=Pakistan, REGION=Middle East, Fare=Low]
Name: TRANS_L1, dtype: object


In [ ]:
# ==== Cell 10: Simple Apriori implementation ====

def apriori(transactions, minsup_count=5, max_len=2):
    all_items = sorted(set(i for t in transactions for i in t))
    freq_itemsets = []

    for k in range(1, max_len + 1):
        for cand in combinations(all_items, k):
            support = sum(1 for t in transactions if set(cand).issubset(t))
            if support >= minsup_count:
                freq_itemsets.append((cand, support))
    return freq_itemsets

In [ ]:
# ==== Cell 11: Mine patterns at Level 1 (REGION + FARE) ====

transactions_L1 = df["TRANS_L1"].tolist()
freq_L1 = apriori(transactions_L1, minsup_count=5, max_len=2)

print("=== Frequent Itemsets – LEVEL 1 (Region + Fare) ===")
for items, sup in freq_L1:
    print(f"Items: {items} | Support Count: {sup}")

=== Frequent Itemsets – LEVEL 1 (Region + Fare) ===
Items: ('Fare=High',) | Support Count: 57
Items: ('Fare=Low',) | Support Count: 55
Items: ('Fare=Medium',) | Support Count: 55
Items: ('REGION=Asia/Others',) | Support Count: 24
Items: ('REGION=Europe/UK',) | Support Count: 23
Items: ('REGION=Middle East',) | Support Count: 136
Items: ('REGION=North America',) | Support Count: 18
Items: ('REGION=Pakistan',) | Support Count: 164
Items: ('Fare=High', 'REGION=Asia/Others') | Support Count: 16
Items: ('Fare=High', 'REGION=Europe/UK') | Support Count: 16
Items: ('Fare=High', 'REGION=Middle East') | Support Count: 29
Items: ('Fare=High', 'REGION=North America') | Support Count: 15
Items: ('Fare=High', 'REGION=Pakistan') | Support Count: 55
Items: ('Fare=Low', 'REGION=Europe/UK') | Support Count: 5
Items: ('Fare=Low', 'REGION=Middle East') | Support Count: 54
Items: ('Fare=Low', 'REGION=Pakistan') | Support Count: 55
Items: ('Fare=Medium', 'REGION=Asia/Others') | Support Count: 6
Items: ('Fa

In [ ]:
# ==== Cell 12: Mine patterns at Level 2 (CITY + FARE) ====

transactions_L2 = df["TRANS_L2"].tolist()
freq_L2 = apriori(transactions_L2, minsup_count=5, max_len=2)

print("=== Frequent Itemsets – LEVEL 2 (City + Fare) ===")
for items, sup in freq_L2:
    print(f"Items: {items} | Support Count: {sup}")

=== Frequent Itemsets – LEVEL 2 (City + Fare) ===
Items: ('CITY=BKK',) | Support Count: 7
Items: ('CITY=DPS',) | Support Count: 6
Items: ('CITY=DXB',) | Support Count: 133
Items: ('CITY=HKT',) | Support Count: 8
Items: ('CITY=JED',) | Support Count: 11
Items: ('CITY=KHI',) | Support Count: 164
Items: ('CITY=LHR',) | Support Count: 10
Items: ('CITY=LON',) | Support Count: 7
Items: ('CITY=MED',) | Support Count: 13
Items: ('CITY=ORD',) | Support Count: 11
Items: ('CITY=YYZ',) | Support Count: 14
Items: ('Fare=High',) | Support Count: 57
Items: ('Fare=Low',) | Support Count: 55
Items: ('Fare=Medium',) | Support Count: 55
Items: ('CITY=BKK', 'CITY=KHI') | Support Count: 7
Items: ('CITY=DPS', 'CITY=DXB') | Support Count: 5
Items: ('CITY=DPS', 'CITY=KHI') | Support Count: 6
Items: ('CITY=DXB', 'CITY=JED') | Support Count: 9
Items: ('CITY=DXB', 'CITY=KHI') | Support Count: 130
Items: ('CITY=DXB', 'CITY=LHR') | Support Count: 9
Items: ('CITY=DXB', 'CITY=MED') | Support Count: 13
Items: ('CITY=